# Data Management&Visualization project

## Scraping and Pruning Data

In [1]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import tmdbsimple as tmdb
from textblob import TextBlob
from limbic.emotion.models.tf_limbic_model import TfLimbicModel
from limbic.emotion.nrc_utils import load_nrc_lexicon
from limbic.emotion.models import LexiconLimbicModel
import spacy
import numpy as np
from datetime import datetime
from Keywords import keywords as kw
pd.options.display.max_colwidth = 100
# API PIETRO tmdb.API_KEY = '3c2aa252cbb71c3ad968ae199ce152ca'
tmdb.API_KEY = '378c352c26eff07972346017b9178331'  
search = tmdb.Search()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['2020', 'al', 'api', 'blog', 'contact', 'couldn', 'didn', 'doesn', 'github', 'hadn', 'hasn', 'haven', 'isn', 'mon', 'pricing', 'privacy', 'security', 'shouldn', 'status', 'terms', 'training', 'vis', 'viser', 'visest', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [13]:
orderList = ['0','A','B','C','D','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']

In [14]:
for order in orderList:
    filmLinks = []
    filmTitle = []
    filmYear = []
    filmScript = []
    idLi=[]
    URL = 'https://www.springfieldspringfield.co.uk/movie_scripts.php?order=' + order + '&page=1'
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    lastPage = int(soup.body.div.find('div',id='content_container').find('div',class_='main-content').find('div',class_='main-content-left').find_all('div',class_='pagination')[0].text.split()[-1])
    print('List: ' + order )
    print('# Pages: ' + str(lastPage))
    for i in range(1, lastPage+1):
        URLinner = 'https://www.springfieldspringfield.co.uk/movie_scripts.php?order=' + order + '&page=' + str(i)
        pageinner = requests.get(URLinner)
        soupinner = BeautifulSoup(pageinner.content, 'html.parser')
        pageLinks = soupinner.body.div.find('div',id='content_container').find('div',class_='main-content').find('div',class_='main-content-left').find_all('a',class_='script-list-item')
        for j in range (0, len(pageLinks)):
            URLdf = 'https://www.springfieldspringfield.co.uk/' + pageLinks[j]['href']
            pagedf = requests.get(URLdf)
            soupdf = BeautifulSoup(pagedf.content, 'html.parser')   
            titleLine = soupdf.body.find('div',class_='wrapper').find('div',id='content_container').find('div',class_='main-content').find('div',class_='main-content-left').find('div',class_='breadcrumbs')
            if(titleLine):
                titleLine = titleLine.text.split('>')[1].strip()
                title = titleLine[:-6]
                year = int(titleLine[-5:-1])
                if year>1960:
                    if re.search(r'[()]+',title):  
                        title=title.split('(')[1].strip(') ')
                    else:
                        title=title.strip()
                    for el in range(0,len(search.movie(query=title,year=year)['results'])):
                        if search.movie(query=title,year=year)['results'][el]['title']==title:
                            idf=int(search.movie(query=title,year=year)['results'][el]['id'])
                            script = soupdf.body.find('div',class_='wrapper').find('div',id='content_container').find('div',class_='main-content').find('div',class_='main-content-left').find('div',class_='movie_script').find('div',class_='scrolling-script-container').text.strip('\n')       
                            filmLinks.append(URLdf)
                            filmTitle.append(title)
                            filmYear.append(year)
                            filmScript.append(script)
                            idLi.append(idf)
        print('Page: ',i)
    df = pd.DataFrame(filmTitle,columns=['filmTitle'])
    df['filmYear'] = filmYear
    df['filmLinks'] = filmLinks
    df['filmScript'] = filmScript
    df['TMDBID'] = idLi
    df.to_csv(r'Data\\Data'+str(order)+'.csv',index=False)

List: C
# Pages: 94
Page:  1


KeyboardInterrupt: 

## Sentiment and Text analysis

In [2]:
tf_model = TfLimbicModel()
nrc_lexcon_file_path = 'NRC-Sentiment-Emotion-Lexicons\\NRC-Affect-Intensity-Lexicon\\NRC-AffectIntensity-Lexicon.txt'
lexicon = load_nrc_lexicon(nrc_lexcon_file_path, 'affect_intensity')
lb = LexiconLimbicModel(lexicon)
nlp = spacy.load('en_core_web_sm')

In [3]:
delimiters = ".", "?"
regexPattern = '|'.join(map(re.escape, delimiters))
def loadData(path):
    data = pd.read_csv('Data\\Data'+path+'.csv')
    splittedScript = []
    for l in data.filmScript:    
        if isinstance(l, float):
            l = str(l)
            
        split = re.sub(r"[^a-zA-z0-9.?']+",' ',l)
        splittedScript.append(re.split(regexPattern, split))
    data['filmScriptSplitted'] = splittedScript
    data.drop(columns=['filmScript'],inplace=True)
    data.rename(columns={'filmScriptSplitted' : 'Script'}, inplace=True)
    return data

In [5]:
orderList = ['T']
#orderList = ['']
for order in orderList:
    df = loadData(order)
    finaljson = {}
    print('Order: ', order)
    print('Total Scripts: ', df.Script.size)
    for i in range (3000, df.Script.size):
        print('Script: ', i)
        polarity = []
        emotions = []
        ner = []
        cast=[]
        idf=df['TMDBID'][i]
        genres=[]
        for z in tmdb.Movies(idf).info()['genres']:
            genres.append(z['name'])
        budget=tmdb.Movies(idf).info()['budget']
        revenue=tmdb.Movies(idf).info()['revenue']
        vote_average=tmdb.Movies(idf).info()['vote_average']
        vote_count=tmdb.Movies(idf).info()['vote_count']
        for s in tmdb.Movies(idf).credits()['cast']:
            country = tmdb.People(s['id']).info()['place_of_birth']
            if isinstance(country, str):
                country = country.split(' ')[-1]
            popularity = tmdb.People(s['id']).info()['popularity']
            actorInfo={
                'id'          : s['id'],
                'name'        : s['name'],
                'character'   : s['character'],
                'gender'      : s['gender'],
                'country'     : country,
                'popularity'  : popularity,
            }
            cast.append(actorInfo)
        for s in tmdb.Movies(idf).credits()['crew']:
            if s['job']=='Director':
                director=s['name']
        noNullPharases = []
        for l in range(0, len(df.Script[i])):
            if df.Script[i][l] != "" and df.Script[i][l] != " ":
                noNullPharases.append(df.Script[i][l])
                polarity.append(TextBlob(df.Script[i][l]).polarity)
                emotionList = []
                for em in lb.get_sentence_emotions(df.Script[i][l]):
                    emotionInfo = {
                        "Category": em.category,
                        "Value"   : em.value,
                        "Term"    : em.term,
                    }
                    emotionList.append(emotionInfo)
                emotions.append(emotionList)
                doc = nlp(df.Script[i][l])
                nerList = []
                for entity in doc.ents:
                    nerInfo = {
                        "Text" : entity.text,
                        "Label": entity.label_,
                    }
                    nerList.append(nerInfo)
                ner.append(nerList)
        scriptDict = {}
        for n in range(0, len(noNullPharases)):
            sentenceDict ={
                n : {
                    "Sentence" : noNullPharases[n],
                    "Polarity" : polarity[n],
                    "Emotions" : emotions[n],
                    "NER"      : ner[n],               
                }
            }
            scriptDict.update(sentenceDict)
        filmScript = ''.join(map(str, df.Script[i]))
        ##########ADD PREPROCESS DA CORRECTOR###############
        keywords = kw.get_keywords(filmScript)
        companies = tmdb.Movies(idf).info()['production_companies']
        companiesList = []
        for c in companies:
            company = {
                "Name"       : c['name'],
                "Country"    : c['origin_country'], 
            }
            companiesList.append(company)
        countries = tmdb.Movies(idf).info()['production_countries']
        countriesList = []
        for c in countries:
            countries = {
                "Country"       : c['name'],
                "Code"          : c['iso_3166_1'],
            }
            countriesList.append(countries)
        filmJson = {
            i : {
                "Title"      : df.filmTitle[i],
                "Year"       : df.filmYear[i],
                "TMDBID"     : df.TMDBID[i],
                "Budget"     : budget,
                "Revenue"    : revenue,
                "Director"   : director,
                "Genres"     : genres,
                "Cast"       : cast,
                "Vote avg"   : vote_average,
                "Vote count" : vote_count,
                "Companies"  : companiesList,
                "Countries"  : countriesList,
                "Keywords"   : keywords,
                "Script"     : scriptDict,
            }
        }
        finaljson.update(filmJson)
    fdf = pd.DataFrame(finaljson)
    fdf.to_json('JSON\\'+order+'.json')

Order:  T
Total Scripts:  3367
Script:  3000
Script:  3001
Script:  3002
Script:  3003
Script:  3004
Script:  3005
Script:  3006
Script:  3007
Script:  3008
Script:  3009
Script:  3010
Script:  3011
Script:  3012
Script:  3013
Script:  3014
Script:  3015
Script:  3016
Script:  3017
Script:  3018
Script:  3019
Script:  3020
Script:  3021
Script:  3022
Script:  3023
Script:  3024
Script:  3025
Script:  3026
Script:  3027
Script:  3028
Script:  3029
Script:  3030
Script:  3031
Script:  3032
Script:  3033
Script:  3034
Script:  3035
Script:  3036
Script:  3037
Script:  3038
Script:  3039
Script:  3040
Script:  3041
Script:  3042
Script:  3043
Script:  3044
Script:  3045
Script:  3046
Script:  3047
Script:  3048
Script:  3049
Script:  3050
Script:  3051
Script:  3052
Script:  3053
Script:  3054
Script:  3055
Script:  3056
Script:  3057
Script:  3058
Script:  3059
Script:  3060
Script:  3061
Script:  3062
Script:  3063
Script:  3064
Script:  3065
Script:  3066
Script:  3067
Script:  3068
Scr

KeyboardInterrupt: 

In [16]:
#IN CASO DI INTERRUZIONE
lettera = 'D'
fdf = pd.DataFrame(finaljson)
fdf.to_json('JSON\\partial1'+lettera+'.json')